In [1]:
import numpy as np

In [2]:
def pivot(tableau, row, col):
        tableau[row, :] /= tableau[row, col]    
        for r in range(len(tableau)):
            if r != row:
                tableau[r, :] -= tableau[r, col] * tableau[row, :]

In [3]:
def simplex(c, A, b, signs, maximize=True):
    n_vars = len(c)
    n_constraints = len(b)

    tableau = np.zeros((n_constraints + 1, n_vars + n_constraints + 1))

    for i in range(n_constraints):
        tableau[i, :n_vars] = A[i]
        if signs[i] == '<=':
            tableau[i, n_vars + i] = 1  
        elif signs[i] == '>=':
            tableau[i, n_vars + i] = -1 
        else:
            raise ValueError("Invalid sign in constraint")

    tableau[:-1, -1] = b

    if maximize:
        tableau[-1, :n_vars] = -c
    else:
        tableau[-1, :n_vars] = c

    while np.any(tableau[-1, :-1] < 0):
        col = np.argmin(tableau[-1, :-1])
        if np.all(tableau[:-1, col] <= 0):
            raise Exception("Linear program is unbounded.")
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, col], out=np.full_like(tableau[:-1, -1], np.inf), where=tableau[:-1, col] > 0)
        row = np.argmin(ratios)
        pivot(tableau, row, col)

    solution = np.zeros(n_vars)
    for i in range(n_vars):
        if np.sum(tableau[:, i] == 1) == 1 and np.sum(tableau[:, i] == 0) == len(tableau) - 1:
            row = np.where(tableau[:, i] == 1)[0][0]
            solution[i] = tableau[row, -1]

    objective_value = tableau[-1, -1]
    if not maximize:
        objective_value = -objective_value

    return solution, objective_value

In [4]:
c = np.array([1, 2]) 
A = np.array([[1, 1], [-2, 1]])  
b = np.array([10, 4])
signs = ['<=', '<=']

In [5]:
# c = np.array([2, 18])
# A = np.array([[80, 100], [20, 200], [1, 0]])
# b = np.array([100, 300, 1.5])   
# signs = ['>=', '<=', '<=']

In [6]:
solution, objective_value = simplex(c, A, b, signs, maximize=True)
print("Solution:", solution)
print("Objective value:", objective_value)

Solution: [2. 8.]
Objective value: 18.0
